<div align="center">

# Online Advertising Optimization

**A data-driven personal project**  
**Author:** Keshav Asokan  
**Date:** August 09, 2025

---

</div>


## Table of Contents

- [1. Introduction](#1-introduction)
  - [1.1. The Online Advertising Ecosystem](#11-the-online-advertising-ecosystem)
  - [**Key Terminology**](#key-terminology)
  - [1.2. Project Approach](#12-project-approach)
- [2. Data Exploration and Analysis](#2-data-exploration-and-analysis)
  - [2.1. Loading and Examining the Data](#21-loading-and-examining-the-data)
  - [2.2. Calculating Cost-Per-Click (CPC)](#22-calculating-cost-per-click-cpc)
  - [2.3. Initial Observations and Hypotheses](#23-initial-observations-and-hypotheses)
  - [2.4. Interpreting the Visualizations](#24-interpreting-the-visualizations)
  - [2.5. Additional Visualization: Budget vs. Clicks](#25-additional-visualization-budget-vs-clicks)
- [3. Predictive Modeling: Linear Regression for CPC](#3-predictive-modeling-linear-regression-for-cpc)
  - [3.1. Building and Evaluating the Regression Models](#31-building-and-evaluating-the-regression-models)
  - [3.2. Model Agreement with Initial Observations](#32-model-agreement-with-initial-observations)
- [4. Optimization: Finding the Optimal Allocation Strategy](#4-optimization-finding-the-optimal-allocation-strategy)
  - [4.1. Formulating the Linear Program](#41-formulating-the-linear-program)
  - [4.2. Model Implementation with Gurobi](#42-model-implementation-with-gurobi)
  - [**Create the Model**](#create-the-model)
  - [**Define Decision Variables**](#define-decision-variables)
  - [**Add Constraints**](#add-constraints)
  - [**Set the Objective Function**](#set-the-objective-function)
  - [4.3. Solving the Optimization Problem](#43-solving-the-optimization-problem)
  - [4.4. Analysis of the Initial Optimal Strategy](#44-analysis-of-the-initial-optimal-strategy)
- [5. Refining the Model with Diversification](#5-refining-the-model-with-diversification)
  - [5.1. Adding Diversification Constraints](#51-adding-diversification-constraints)
  - [5.2. Analysis of the Diversified Strategy](#52-analysis-of-the-diversified-strategy)
  - [5.3. Sensitivity Analysis](#53-sensitivity-analysis)
- [6. The Economics of Advertising: Optimizing for Revenue](#6-the-economics-of-advertising-optimizing-for-revenue)
  - [6.1. Net Revenue vs. Total Budget](#61-net-revenue-vs-total-budget)
  - [6.2. Financial Insights](#62-financial-insights)
- [7. Finding the Optimal Budget for Maximum Net Revenue](#7-finding-the-optimal-budget-for-maximum-net-revenue)
  - [7.1. Formulating the New Linear Program](#71-formulating-the-new-linear-program)
  - [7.2. Implementing and Solving the Net Revenue Model](#72-implementing-and-solving-the-net-revenue-model)
  - [7.3. Analysis of the Optimal Budget](#73-analysis-of-the-optimal-budget)
  - [7.4. Finding the Break-Even Budget](#74-finding-the-break-even-budget)
- [8. Practical Considerations and Future Work](#8-practical-considerations-and-future-work)
  - [8.1. Additional Metrics for Prediction](#81-additional-metrics-for-prediction)
  - [8.2. Alternative Campaign Objectives](#82-alternative-campaign-objectives)
  - [8.3. Handling Dynamic Keyword Trends](#83-handling-dynamic-keyword-trends)
  - [8.4. Addressing Click Attribution](#84-addressing-click-attribution)
- [9. Optional Advanced Model: Piecewise Linear CPC](#9-optional-advanced-model-piecewise-linear-cpc)
  - [9.1. Implementing the Piecewise Linear Model](#91-implementing-the-piecewise-linear-model)
  - [9.2. Analysis of the Piecewise Linear Model Result](#92-analysis-of-the-piecewise-linear-model-result)

In [ ]:
# Visual/style setup (kept lightweight; safe defaults)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# If seaborn is installed, set a clean theme for nicer plots
try:
    import seaborn as sns
    sns.set_theme(style="whitegrid")
except Exception:
    pass

# Matplotlib defaults
plt.rcParams.update({
    "figure.figsize": (9, 6),
    "axes.titlesize": 14,
    "axes.labelsize": 12,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "legend.fontsize": 10,
    "figure.autolayout": True,
})

# Pandas display
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)
pd.options.display.float_format = "{:,.3f}".format

print("✅ Styling and display preferences set.")

# Project: Optimizing an Online Advertising Campaign

## 1. Introduction

This project explores strategies for optimizing online advertising for a high-street fashion retailer in the UK. The retail sector is currently experiencing a significant shift, with in-store sales declining by approximately 4% annually and online sales growing by 10% per year. Despite recent investments in a modern e-commerce platform, the brand's online presence is still developing, and website traffic remains limited.

The primary objective is to design and evaluate a large-scale online advertising campaign to increase platform visibility and attract new visitors. The campaign will operate with a minimum budget of £1 million over one month and aims to acquire at least 200,000 new visitors. To achieve this, this analysis applies data-driven optimization techniques to develop actionable recommendations for the campaign strategy.

### 1.1. The Online Advertising Ecosystem

The online advertising ecosystem primarily involves two key intermediaries:

<img src="advertising.png" width="500">

- **Online Ad Exchanges**: These platforms, like Google Ads and Facebook, run auctions to allocate advertising space to advertisers. They determine the auction mechanism—deciding which ads are shown and at what price.
- **Demand-Side Platforms (DSPs)**: DSPs manage the advertising campaigns on behalf of clients. They are responsible for the operational execution, deciding whether to participate in auctions and determining appropriate bid amounts.

For this project, we've engaged a DSP to run a campaign on Google Ads. However, several key parameters need to be determined. This analysis will address the following fundamental questions:

- Which keywords should be targeted for advertising?
- How should the budget be allocated across these keywords?
- What is the optimal total budget for the advertising campaign?

#### **Key Terminology**
When the DSP wins an auction, the ad is displayed to the user, which is known as an **impression**. A crucial metric for advertisers is the **Cost-Per-Click (CPC)**, representing the budget spent per click. The CPC for a given keyword, $k$, is calculated as:

$$ CPC(k) = \frac{{\text{Amount spent on keyword } k}}{{\text{Number of clicks on keyword } k}} $$

### 1.2. Project Approach

This project uses historical data provided by the DSP to inform decisions. The data comes from advertising campaigns conducted for similar clients—competing UK retailers with the same market positioning. Each data point represents a keyword and a specific spending level, along with the resulting number of clicks. Since each click directs a user to the e-commerce site, we can approximate one click as one new visitor.

The goal is to build a model that determines the optimal spending for each keyword, given a prospective budget of £1 million. The process involves:
1.  **Data Exploration**: Analyzing historical campaign data to understand keyword performance.
2.  **CPC Modeling**: Building a model to predict the Cost-Per-Click for each keyword.
3.  **Optimization**: Using a linear programming model to determine the optimal budget allocation.

This data-driven approach will lead to a set of actionable recommendations for the advertising campaign.

## 2. Data Exploration and Analysis

The DSP has provided data on five keywords selected by the marketing team. For each keyword, the DSP has identified previous campaigns from similar UK retailers, showing various monthly spending levels (from thousands to millions of pounds) and the corresponding number of clicks generated. The campaign data is reported on a monthly basis.

### 2.1. Loading and Examining the Data

First, let's load the data and get a general overview of its structure and summary statistics.

In [ ]:
# Import pandas for data manipulation
import pandas as pd

# Load the dataset
df = pd.read_csv("data_keywords.csv").set_index("ID")

In [ ]:
# Display the first 5 rows
print("--- First 5 Rows ---")
print(df.head())

# Display summary statistics
print("\n--- Summary Statistics ---")
print(df.describe())

# Calculate mean budget and clicks by keyword
print("\n--- Mean Values by Keyword ---")
print(df.groupby('keyword').mean())

# Calculate median budget and clicks by keyword
print("\n--- Median Values by Keyword ---")
print(df.groupby('keyword').median())

### 2.2. Calculating Cost-Per-Click (CPC)

Now, we'll calculate the CPC for each campaign and add it as a new column to our dataframe.

In [ ]:
# Calculate and add the CPC column
df['CPC'] = df['budget'] / df['clicks']

Next, let's visualize the distribution of CPC for each keyword using a boxplot. This will help us understand the cost-effectiveness and variability of each keyword.

In [ ]:
# Import relevant libraries for plotting
import seaborn as sns
import matplotlib.pyplot as plt

# Set up plot environment
%matplotlib inline
plt.subplots(figsize=(9,6))
sns.set(style="ticks", palette="pastel")

# Create a boxplot to show CPC distribution by keyword
sns.boxplot(x="keyword", y="CPC", data=df)

# Finalize plot display
sns.despine(offset=10, trim=True)

### 2.3. Initial Observations and Hypotheses

Based on the boxplot, **'retail UK'** appears to have the best (i.e., cheapest) CPC, while **'dua lipa gloves'** has the worst (most expensive) CPC.

The variability in CPC for the same keyword could be due to several factors:

- **Ad Quality**: Higher quality and more relevant ads often achieve better CPCs.
- **Platform Price Discrimination**: Ad exchanges might offer discounts to certain buyers based on their relationship or volume.
- **Auction Dynamics**: Different valuation methods used by ad exchanges can lead to varying prices.

Let's refine the visualization to see if budget size affects CPC.

In [ ]:
# Define a 'large-budget' category for budgets >= £20,000
df['large-budget'] = df['budget'] >= 20000

# Set up the plot
plt.subplots(figsize=(10,7))
sns.set(style="ticks", palette="pastel")

# Create a grouped boxplot to compare CPC for large vs. small budgets
sns.boxplot(x="keyword", y="CPC", hue="large-budget", palette=["m", "g"], data=df)

# Finalize plot display
sns.despine(offset=10, trim=True)

In [ ]:
# Set seaborn style for regression plot
sns.set()

# Get a list of unique keywords
keywords = df.keyword.drop_duplicates().tolist()
print(f"Keywords being analyzed: {keywords}")

# Plot the relationship between clicks and budget for the 'cardigan' keyword
ax = sns.regplot(x="clicks", y="budget", data=df[df["keyword"] == "cardigan"], order=2)
plt.title('Budget vs. Clicks for "cardigan" Keyword')

### 2.4. Interpreting the Visualizations

The analysis separates budgets into 'small' (< £20k) and 'large' (>= £20k).

- **For small budgets**, 'dua lipa gloves' has the lowest median CPC, while 'spotted dress' has the highest.
- **For large budgets**, 'free shipping UK' shows the lowest median CPC, while 'cardigan' has the highest. The wide interquartile ranges for 'cardigan' and 'dua lipa gloves' suggest high CPC variability for these keywords. Notably, 'retail UK' has no data for large budgets.

The regression plot shows a positive, non-linear relationship between budget and clicks, with some high-budget outliers indicating that while most campaigns have smaller spends, a few are major players.

### 2.5. Additional Visualization: Budget vs. Clicks

To further explore the relationship, let's examine the budget-click relationship for another keyword.

In [ ]:
# Plot the relationship between clicks and budget for the 'dua lipa gloves' keyword
ax = sns.regplot(x="clicks", y="budget", data=df[df["keyword"] == "dua lipa gloves"], order=2)
plt.title('Budget vs. Clicks for "dua lipa gloves" Keyword')

For the 'dua lipa gloves' keyword, we again observe a positive, non-linear relationship between budget and clicks. The concentration of data points at lower budget levels, with a few high-budget outliers, is also apparent here.

## 3. Predictive Modeling: Linear Regression for CPC

While the relationship between clicks and budget is not perfectly linear, a linear model provides a solid first-order approximation. Here, we will build a simple linear regression model to predict the number of clicks (Y) as a function of the budget spent (X) for each keyword.

The model is defined as:
$$ \text{numberOfClicks} = \alpha \cdot \text{budgetSpent} $$

*Note: We assume a zero-intercept model, as spending £0 should result in 0 clicks. The ultimate goal is to create a dictionary mapping each keyword to its predicted CPC value.*

### 3.1. Building and Evaluating the Regression Models

The following code iterates through each keyword, fits a linear regression model, and calculates the corresponding CPC. The code is commented to explain each step.

In [ ]:
# Import necessary libraries
import numpy as np
from sklearn.linear_model import LinearRegression

# Create a list of unique keywords
keywords = df.keyword.drop_duplicates().tolist()

# Initialize an empty dictionary to store the calculated CPC for each keyword
CPC = {}

# Loop through each keyword to build a separate regression model
for k in keywords:
    # Filter the dataframe for the current keyword
    df_k = df[df["keyword"] == k]
    
    # Prepare the independent variable (budget) for scikit-learn
    # .values extracts the numpy array, .reshape ensures it's a 2D array
    X = df_k.budget.values.reshape(-1, 1)
    
    # Prepare the dependent variable (clicks)
    y = df_k.clicks.values
    
    # Instantiate a linear regression model without an intercept
    reg = LinearRegression(fit_intercept=False)
    
    # Fit the model to the data for the current keyword
    reg.fit(X, y)
    
    # The model gives clicks = alpha * budget, so alpha = reg.coef_[0]
    # Since CPC = budget / clicks, CPC = 1 / alpha
    CPC[k] = 1.0 / reg.coef_[0]
    
    # Display the calculated CPC for the keyword
    print(f"\nThe predicted CPC of keyword '{k}' is: £{CPC[k]:.2f} per click")
    
    # Display the R-squared value to evaluate model fit
    print(f"The in-sample R-squared for this model is: {reg.score(X, y):.4f}")

### 3.2. Model Agreement with Initial Observations

Yes, the predictive model's results align with the initial observations from the data exploration. The keyword **'retail UK'** has the lowest predicted CPC, making it the most cost-effective, while **'dua lipa gloves'** has the highest, confirming it is the most expensive. The ranking of keywords by CPC is consistent with what was seen in the boxplots.

The very high predicted CPC for 'dua lipa gloves' and its negative R-squared value suggest that a simple linear model is a poor fit for this keyword, likely due to high variability or a non-linear relationship not captured by this model.

## 4. Optimization: Finding the Optimal Allocation Strategy

Using the CPC values derived from our model, we will now determine how to allocate a £1M budget across the keywords to maximize the total number of clicks.

### 4.1. Formulating the Linear Program

The optimal allocation problem can be formulated as a linear program with the following components:

- **Decision Variables**: Let $x_k$ be the amount of money (in £) to be spent on keyword $k$.
- **Objective Function**: Maximize the total number of clicks. Since the number of clicks for a keyword is $\frac{1}{CPC_k} \cdot x_k$, the objective is to maximize: 
  $$ \sum_{k \in \text{keywords}} \frac{1}{CPC_k} \cdot x_k $$
- **Constraints**: 
  1. The total budget spent cannot exceed £1,000,000:
     $$ \sum_{k \in \text{keywords}} x_k \le 1,000,000 $$
  2. The budget allocated to each keyword must be non-negative:
     $$ x_k \ge 0 \quad \forall k \in \text{keywords} $$

### 4.2. Model Implementation with Gurobi

In [ ]:
# Import the Gurobi library and its functions
import gurobipy as gp
from gurobipy import GRB, quicksum

#### **Create the Model**

In [ ]:
# Create a new Gurobi model named 'allocation'
m = gp.Model("allocation")

#### **Define Decision Variables**

In [ ]:
# Create a decision variable for each keyword, representing the budget allocated.
# The lower bound (lb) is 0, as we cannot spend a negative amount.
allocation = m.addVars(keywords, lb=0, name="allocation")

This code creates a set of decision variables, `allocation[k]`, for each keyword `k`. These variables will hold the optimal budget split determined by the solver. For example, if the optimal solution involves no spending on 'cardigan', the value of `allocation['cardigan']` will be 0.

#### **Add Constraints**
The primary constraint is the total budget limit.

In [ ]:
# Add the budget constraint: the sum of allocations for all keywords must not exceed £1,000,000.
budget_constraint = m.addConstr(quicksum(allocation[k] for k in keywords) <= 1000000, "budget_constraint")

#### **Set the Objective Function**
The objective is to maximize the total number of clicks.

In [ ]:
# Set the objective to maximize the sum of (allocation / CPC) for all keywords.
# This is equivalent to maximizing total clicks.
m.setObjective(quicksum(1.0/CPC[k] * allocation[k] for k in keywords), GRB.MAXIMIZE)

### 4.3. Solving the Optimization Problem

With the model formulated, we can now solve it to find the optimal allocation.

In [ ]:
# Define a function to print the solution in a readable format
def print_solution():
    if m.status == GRB.OPTIMAL:
        print(f'\nOptimal Clicks: {m.objVal:,.0f}')
        print('\nOptimal Allocation:')
        allocation_x = m.getAttr('x', allocation)
        for k in keywords:
            if allocation_x[k] > 0:
                print(f"  - {k}: £{allocation_x[k]:,.0f}")
    else:
        print(f'No optimal solution found. Status code: {m.status}')

# Optimize the model
m.optimize()

# Print the results
print_solution()

### 4.4. Analysis of the Initial Optimal Strategy

The model recommends allocating the entire **£1,000,000 budget to a single keyword: 'retail UK'**. This is because 'retail UK' has the lowest CPC, and the linear model assumes this CPC remains constant regardless of the budget. Therefore, to maximize clicks, the model logically invests everything in the most cost-effective option.

This strategy is only realistic under specific conditions, such as low competition for the keyword and the absence of diminishing returns (i.e., the CPC does not increase as we spend more). In reality, this approach is risky as it ignores potential market saturation and the risk of **ad fatigue**, where users become overexposed to an ad and stop paying attention, reducing its effectiveness.

## 5. Refining the Model with Diversification

The initial result of allocating the entire budget to one keyword is a common outcome for simple linear optimization models. To create a more realistic and robust strategy, it's wise to introduce diversification. Let's impose a constraint that no more than **£300,000** can be invested in any single keyword. This will force the model to spread the budget across multiple keywords.

### 5.1. Adding Diversification Constraints

We will add new constraints to our existing model.

In [ ]:
# Add a constraint for each keyword to limit its allocation to a maximum of £300,000.
diversification_constrs = m.addConstrs((allocation[k] <= 300000 for k in keywords), "diversification_constraint")

Now, let's re-run the optimization with these new constraints.

In [ ]:
# Re-optimize the model with the new diversification constraints
m.optimize()

# Print the new optimal solution
print_solution()

### 5.2. Analysis of the Diversified Strategy

With the diversification constraint, the model now recommends splitting the £1M budget across four keywords:

- **£300,000** for 'retail UK'
- **£300,000** for 'free shipping UK'
- **£300,000** for 'spotted dress'
- **£100,000** for 'cardigan'

The keyword **'dua lipa gloves'** is excluded entirely because its CPC is significantly higher than the others, making it an inefficient choice for maximizing clicks even with the diversification rule.

This allocation is more practical as it reduces risk by not relying on a single keyword and targets a broader audience.

### 5.3. Sensitivity Analysis

How many more clicks could be generated by slightly increasing the £300,000 diversification limit? We can determine this by examining the **shadow price** (dual value) of the diversification constraints. The shadow price tells us the change in the objective function (total clicks) for a one-unit (£1) increase in the constraint's right-hand side.

In [ ]:
# Perform sensitivity analysis on the diversification constraints
print("--- Shadow Prices of Diversification Constraints ---")
for k in keywords:
    if diversification_constrs[k].Pi != 0:
      print(f"The shadow price for '{k}' is: {diversification_constrs[k].Pi:.4f}")

The shadow price for 'retail UK' is approximately 0.22. This means that for every £1 increase in its budget cap (above £300k), we could generate an additional 0.22 clicks, assuming the budget is taken from a less efficient keyword like 'cardigan'. This indicates that 'retail UK' is the most constrained and most valuable keyword.

## 6. The Economics of Advertising: Optimizing for Revenue

Let's shift the focus from simply maximizing clicks to understanding the financial outcomes. Assume each click is monetized at an average of **£4**. The key financial metric is the net revenue:

$$ \text{Net Revenue} = \text{Total Revenue} - \text{Total Budget} $$

### 6.1. Net Revenue vs. Total Budget

We will now plot the optimal net revenue as a function of the total budget. This involves running our diversified optimization model for various budget levels from £0 to £1M.

In [ ]:
# Define a range of budgets from £0 to £1M in increments of £100K
budgets = np.arange(0, 1100000, 100000)

# Initialize an array to store the corresponding optimal net revenue
net_revenue = np.zeros(len(budgets))

# Loop through each budget value to run the optimization
for i, budget_val in enumerate(budgets):
    # Deactivate Gurobi logs for cleaner output
    m.setParam('OutputFlag', 0)

    # Update the right-hand side of the budget constraint
    budget_constraint.setAttr("rhs", budget_val)
    
    # Re-optimize the model with the new budget
    m.optimize()
    
    # Calculate and store the optimal net revenue
    # Net Revenue = (Total Clicks * £4) - Budget
    if m.status == GRB.OPTIMAL:
        net_revenue[i] = (m.objVal * 4) - budget_val

# Restore Gurobi logs
m.setParam('OutputFlag', 1)

In [ ]:
# Plot the net revenue as a function of the budget
plt.figure(figsize=(10, 6))
ax = sns.scatterplot(x=budgets, y=net_revenue)
sns.lineplot(x=budgets, y=net_revenue, ax=ax)
ax.set_xlabel("Total Budget (£)")
ax.set_ylabel("Net Revenue (£)")
plt.title('Optimal Net Revenue vs. Total Budget')
plt.grid(True)

### 6.2. Financial Insights

The plot reveals a critical insight: net revenue becomes negative after the budget surpasses approximately **£300,000**. The deficit continues to grow as the budget increases, making the campaign progressively more unprofitable. This suggests that if the primary goal is to maximize net revenue, the total budget should be carefully managed and potentially capped around this point of diminishing returns.

## 7. Finding the Optimal Budget for Maximum Net Revenue

Instead of testing discrete budget levels, we can use linear programming to directly find the optimal budget that maximizes net revenue. We'll modify our model to treat the total budget as a decision variable.

### 7.1. Formulating the New Linear Program

The model is similar to the previous one, but with a new variable and a different objective.

- **Decision Variables**:
  - $x_k$: The amount of money (in £) to be spent on keyword $k$.
  - $B$: The total budget for the campaign.
- **Objective Function**: Maximize the net revenue.
  $$ \text{Maximize} \quad \left( \sum_{k \in \text{keywords}} \frac{4}{CPC_k} \cdot x_k \right) - B $$
- **Constraints**:
  1. The sum of allocations cannot exceed the total budget:
     $$ \sum_{k \in \text{keywords}} x_k \le B $$
  2. Diversification: The budget for each keyword cannot exceed £300,000:
     $$ x_k \le 300,000 \quad \forall k \in \text{keywords} $$
  3. All variables must be non-negative.

### 7.2. Implementing and Solving the Net Revenue Model

In [ ]:
# Create a new model for optimizing net revenue
m_rev = gp.Model("allocation_budget")

# Define decision variables
allocation_rev = m_rev.addVars(keywords, lb=0, name="allocation")
budget_rev = m_rev.addVar(lb=0, name='budget')

# Add budget constraint
m_rev.addConstr(quicksum(allocation_rev[k] for k in keywords) <= budget_rev, "budget_constraint")

# Add diversification constraints
m_rev.addConstrs((allocation_rev[k] <= 300000 for k in keywords), "diversification_constraint")

# Set the objective to maximize net revenue
m_rev.setObjective(quicksum(4.0/CPC[k] * allocation_rev[k] for k in keywords) - budget_rev, GRB.MAXIMIZE)

# Optimize the model
m_rev.optimize()

# Print the solution
if m_rev.status == GRB.OPTIMAL:
    print(f'\nMaximum Net Revenue: £{m_rev.objVal:,.2f}')
    print(f"\nOptimal Budget: £{budget_rev.x:,.2f}")
    print('\nOptimal Allocation:')
    allocation_x_rev = m_rev.getAttr('x', allocation_rev)
    for k in keywords:
        if allocation_x_rev[k] > 0:
            print(f"  - {k}: £{allocation_x_rev[k]:,.2f}")
else:
    print(f'No optimal solution found. Status code: {m_rev.status}')

### 7.3. Analysis of the Optimal Budget

To maximize net revenue, the optimal budget is **£300,000**, which should be fully invested into the keyword **'retail UK'**. This confirms our finding from the sensitivity plot: spending beyond this point on less efficient keywords leads to a decrease in overall profitability. The model invests up to the diversification limit in the single most profitable keyword and stops, as adding budget for other keywords would result in a net loss.

### 7.4. Finding the Break-Even Budget

What if the goal is to spend the largest possible budget while ensuring the campaign breaks even (i.e., net revenue is zero)? We can modify the LP to find this.

In [ ]:
# Use the same model, but change the objective and add a breakeven constraint
m_rev.addConstr((quicksum(4.0/CPC[k] * allocation_rev[k] for k in keywords) - budget_rev) == 0, "breakeven_constraint")
m_rev.setObjective(budget_rev, GRB.MAXIMIZE) # Now, maximize the budget itself

# Re-optimize the model
m_rev.optimize()

# Print the solution
if m_rev.status == GRB.OPTIMAL:
    print(f'\nMaximum Break-Even Budget: £{m_rev.objVal:,.2f}')
    print('\nAllocation at Break-Even:')
    allocation_x_rev = m_rev.getAttr('x', allocation_rev)
    for k in keywords:
        if allocation_x_rev[k] > 1:
          print(f"  - {k}: £{allocation_x_rev[k]:,.2f}")
else:
    print(f'No optimal solution found. Status code: {m_rev.status}')

The maximum budget for a break-even campaign is **£313,345**. This is achieved by allocating £300,000 to 'retail UK' and the remaining £13,345 to 'free shipping UK'. This budget is slightly higher than the one for maximum net revenue, demonstrating the trade-off between maximizing profit and maximizing reach (budget).

## 8. Practical Considerations and Future Work

While the linear programming model provides valuable insights, it's important to consider its limitations and potential areas for improvement.

### 8.1. Additional Metrics for Prediction

To build a more comprehensive model, it would be useful to predict other key performance indicators (KPIs) such as:

- **Click-Through Rate (CTR)**: The ratio of clicks to impressions. A high CTR indicates ad relevance.
- **Bounce Rate**: The percentage of visitors who leave the site after viewing only one page. A high bounce rate suggests a mismatch between ad content and landing page experience.
- **Conversion Rate**: The percentage of visitors who complete a desired action (e.g., make a purchase). This is the ultimate measure of campaign success.

### 8.2. Alternative Campaign Objectives

Instead of maximizing clicks, a more sophisticated objective would be to maximize:

- **Customer Lifetime Value (CLV)**: Focus on acquiring customers who are likely to make repeat purchases and have a higher long-term value to the business.
- **Engagement**: Target users who are more likely to spend significant time on the site, indicating genuine interest.

### 8.3. Handling Dynamic Keyword Trends

Keyword popularity changes rapidly. Relying on historical data for trending keywords (like 'Dua Lipa gloves') is challenging.

- **Issue**: Budget planning is difficult when keyword performance is volatile. The backend infrastructure also needs to adapt quickly.
- **Strategy**: Earmark a portion of the budget for experimental or trending keywords. Use automated tools or a digital marketing agency to monitor trends and adjust the keyword strategy dynamically.

### 8.4. Addressing Click Attribution

When advertising across multiple platforms, a user may see an ad several times before clicking. This creates an attribution problem.

- **Issue**: It's difficult to determine which specific ad or platform was responsible for the final click, making it hard to measure the true ROI of each channel.
- **Strategy**: Implement a more advanced attribution model (e.g., multi-touch attribution) instead of a simple last-click model. This would provide a more accurate picture of how different keywords and platforms contribute to conversions.

## 9. Optional Advanced Model: Piecewise Linear CPC

The linear CPC model is a simplification. In reality, the cost per click is likely a non-linear function of the budget; getting 1 million clicks is probably disproportionately more expensive than getting 1,000 clicks. The heatmap below, showing CPC per keyword and budget level, supports this intuition.

<img src="CPC.png" width="800">

The heatmap shows that for most keywords, CPC increases as the budget level grows. This is due to the law of diminishing returns: as you spend more, you have to bid higher or target less relevant audiences to get additional clicks, driving up the average cost.

To account for this, we can approximate the non-linear CPC using a **piecewise linear function**.

In [ ]:
# This dictionary defines the slopes (CPC) for two different budget segments (pieces) for each keyword.
# Format: { (keyword, piece): [CPC, lower_bound, upper_bound] }

pieces_list = ['piece A', 'piece B']
PiecewiseCPC = {
    ('retail UK', 'piece A'): [4.1, 0, 20000],
    ('retail UK', 'piece B'): [1e+100, 20000, 1000000], # Large value to act as a cap
    ('free shipping UK', 'piece A'): [3.0, 0, 20000],
    ('free shipping UK', 'piece B'): [6.9, 20000, 1000000],
    ('cardigan', 'piece A'): [1.5, 0, 20000],
    ('cardigan', 'piece B'): [41.1, 20000, 1000000],
    ('dua lipa gloves', 'piece A'): [2.5, 0, 20000],
    ('dua lipa gloves', 'piece B'): [233.6, 20000, 1000000],
    ('spotted dress', 'piece A'): [4.7, 0, 20000],
    ('spotted dress', 'piece B'): [10.4, 20000, 1000000]
}

### 9.1. Implementing the Piecewise Linear Model

To implement this, we'll create separate decision variables for each piece of the CPC function for each keyword. For example, the allocation for 'retail UK' will be $x_{\text{retail UK}} = y_{\text{retail UK, A}} + y_{\text{retail UK, B}}$, where $y_{\text{retail UK, A}}$ is the budget spent in the first segment (up to a spend corresponding to 20,000 clicks) and $y_{\text{retail UK, B}}$ is the budget for the second segment.

In [ ]:
# Prepare data structures for the piecewise model
piecewise_vars = list(PiecewiseCPC.keys())
slopes = {key: val[0] for key, val in PiecewiseCPC.items()}
upper_bounds = {key: val[2] for key, val in PiecewiseCPC.items()}

# Create a new Gurobi model for the piecewise linear problem
m_plc = gp.Model("PiecewiseLinearCost")

# Add decision variables for each piece of each keyword
allocation_plc = m_plc.addVars(piecewise_vars, lb=0, name="PLC_allocation")

# Add constraints for the upper bound of each piece's budget
for k, p in piecewise_vars:
    piece_click_limit = upper_bounds[(k,p)]
    # Budget limit = clicks * CPC
    if p == 'piece A':
      budget_limit_A = piece_click_limit * slopes[(k,p)]
      m_plc.addConstr(allocation_plc[(k, p)] <= budget_limit_A, f"upper_bound_{k}_{p}")

# Add the total budget constraint
m_plc.addConstr(quicksum(allocation_plc[j] for j in piecewise_vars) <= 1000000, "total_budget")

# Set the objective to maximize total clicks
m_plc.setObjective(quicksum(allocation_plc[j] / slopes[j] for j in piecewise_vars), GRB.MAXIMIZE)

# Optimize the model
m_plc.optimize()

# Print the solution
if m_plc.status == GRB.OPTIMAL:
    print(f'\nOptimal Clicks (Piecewise Model): {m_plc.objVal:,.0f}')
    print('\nOptimal Allocation:')
    allocation_x_plc = m_plc.getAttr('x', allocation_plc)
    for k in keywords:
      total_alloc = allocation_x_plc[(k, 'piece A')] + allocation_x_plc[(k, 'piece B')]
      if total_alloc > 0:
        print(f"  - {k}: £{total_alloc:,.0f}")
else:
    print(f'No optimal solution found. Status code: {m_plc.status}')

### 9.2. Analysis of the Piecewise Linear Model Result

The piecewise linear model provides a more nuanced allocation. Unlike the simple linear model which focused on a single keyword, this model distributes the budget more broadly. It allocates funds to the most cost-effective initial segments ('piece A') of several keywords before moving to the more expensive secondary segments ('piece B').

This strategy is far more realistic, as it inherently captures the diminishing returns of advertising spend and leads to a naturally diversified portfolio that balances cost and reach, without needing an arbitrary diversification cap.

## Project Summary

**What I built:** A full pipeline from exploratory analysis to predictive modeling and linear-programming-based budget allocation for online ads, with an optional piecewise CPC extension.

**Key takeaways**
- CPC varies widely across keywords; simple linear modeling is a useful starting point but can miss diminishing returns.
- A naive LP will concentrate spend on the single cheapest CPC; diversification constraints or non-linear CPC fix this.
- From a **revenue** lens, an optimal budget often caps far **below** the maximum available spend.

**Next steps**
- Add conversion data to optimize for **profit** or **LTV**, not just clicks.
- Replace linear CPC with a calibrated piecewise or nonlinear response curve learned from data.
- Introduce uncertainty (e.g., CPC bands) and solve a **robust** or **stochastic** optimization variant.
